In [2]:
# 회귀분석
# 기준값 : 분산분석에서의 F통계량, 분산의 감소량
# > SSE(Sum of Sequred Error, 분산의 감소량)이 최대일 때 나눠준다.

In [8]:
import pandas as pd
df = pd.read_csv('data/kc_house_data.csv')

In [5]:
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,standard,0,3,7,1180,0,1955,0,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,standard,0,3,7,2170,400,1951,1991,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,standard,0,3,6,770,0,1933,0,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,standard,0,5,7,1050,910,1965,0,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,standard,0,3,8,1680,0,1987,0,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,standard,0,3,8,1530,0,2009,0,1530,1509
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,standard,0,3,8,2310,0,2014,0,1830,7200
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,standard,0,3,7,1020,0,2009,0,1020,2007
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,standard,0,3,8,1600,0,2004,0,1410,1287


In [9]:
df = df.drop(['id','date'], axis = 1 )
X = df.drop('price', axis = 1)
y = df['price']

In [10]:
X.info() # object > 연속형으로 변환, One-hot-encoding

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   bedrooms       21613 non-null  int64  
 1   bathrooms      21613 non-null  float64
 2   sqft_living    21613 non-null  int64  
 3   sqft_lot       21613 non-null  int64  
 4   floors         21613 non-null  float64
 5   waterfront     21613 non-null  object 
 6   view           21613 non-null  int64  
 7   condition      21613 non-null  int64  
 8   grade          21613 non-null  int64  
 9   sqft_above     21613 non-null  int64  
 10  sqft_basement  21613 non-null  int64  
 11  yr_built       21613 non-null  int64  
 12  yr_renovated   21613 non-null  int64  
 13  sqft_living15  21613 non-null  int64  
 14  sqft_lot15     21613 non-null  int64  
dtypes: float64(2), int64(12), object(1)
memory usage: 2.5+ MB


In [11]:
X = pd.get_dummies(data = X, columns = ['waterfront'])

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 2021) # 연속형이기 때문에 층화추출은 할 수없다.

In [18]:
# 매개변수 없이 진행
from sklearn.tree import DecisionTreeRegressor
dt_reg = DecisionTreeRegressor()
dt_reg.fit(X_train, y_train)
print(dt_reg.score(X_train, y_train)) # 거의 1에 가깝다.
print(dt_reg.score(X_test, y_test)) # train과 차이가 많다.

0.9993379050282394
0.5448737203359976


In [19]:
dt_reg.get_depth()

40

In [40]:
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth':range(1,40,2), 'min_samples_split':range(2,40,2)}
model_grid_tree = GridSearchCV(DecisionTreeRegressor(), param_grid)

In [41]:
model_grid_tree.fit(X_train, y_train)

GridSearchCV(estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': range(1, 40, 2),
                         'min_samples_split': range(2, 40, 2)})

In [43]:
model_grid_tree.best_estimator_

DecisionTreeRegressor(max_depth=7, min_samples_split=16)

In [44]:
model_grid_tree.score(X_train, y_train)

0.7422795151739846

In [45]:
model_grid_tree.score(X_test, y_test)

0.6921699230139526

In [46]:
y_pred = model_grid_tree.predict(X_test)

In [47]:
resid = y_test - y_pred

In [49]:
import numpy as np
sse = np.sum(resid**2)

In [52]:
MSE = sse/len(y_test)

In [53]:
RMSE = np.sqrt(MSE)

In [54]:
RMSE

202226.2804014401

In [57]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)
mse
np.sqrt(mse)

202226.2804014401

In [58]:
# 설명
# 1. 의사결정나무의 정확도 R^2 값은 0.69으로 모델이 전체 데이터의 70%를 설명할 수 있다.
# 2. RMSE의 값은 202,226이므로, 집값의 평균적인 차이는 202,226만큼 난다고 할 수 있다.


In [59]:
# 의사결정나무는 시각화를 통해 설명력이 높은지 알 수 있다.(기본적으로 의사결정나무는 설명력이 좋다.)